In [ ]:

#这个是对的
import os
import nltk
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import random
from nltk.corpus import stopwords
import operator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

from sklearn.feature_extraction.text import CountVectorizer


stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['/','(',')','{','}','@','|',';','\n','#','+','_','.',',','``',"''",':','-']
stopwords.extend(new_stopwords)

lemmatizer = nltk.stem.WordNetLemmatizer()


path = os.getcwd() + "\\bbc"
list_catagory_files = os.listdir(path)
print(list_catagory_files)
num_catagory = len(list_catagory_files)



def text_label_list(cata_path):
    label = 0
    all_text_list = []
    all_label_list = []
    catagory_file_num = []
    
    for catagory in list_catagory_files:
        catagory_file = path + "\\" + catagory
        print(catagory_file)
        if catagory_file.endswith('.TXT'):
            continue
        else:
        
            file_num = len(os.listdir(catagory_file))
            catagory_file_num.append(file_num)
            print(file_num)
            print(catagory)
            for i in range(1,file_num + 1):
                all_label_list.append(label)
                j = "%03d" % i
                with open("\\".join([catagory_file, str(j) + ".txt"])) as fp:
                    lines = fp.read()
                    article = []
                    text_ = ""
                    text_ += lines
                    article.append(text_)    
                        
                        
                    all_text_list.append(article)
        label += 1
    return all_text_list, all_label_list, catagory_file_num

all_text_list_, all_label_list_, catagory_file_num = text_label_list(list_catagory_files)



        
# get full dataset in order to split our dataset into training and test
dataset_full=[] 
for text, label in zip(all_text_list_, all_label_list_):
    dataset_full.append((text,label))
        

def get_train_test_split(dataset_full,ratio):
    pre_train_set=[]
    pre_test_set=[]
    
    size_dataset_full = len(dataset_full)
    test_size = int(round(size_dataset_full * ratio))
    list_test_indices=random.sample(range(size_dataset_full), test_size)
    
    for i,text in enumerate(dataset_full):
        if i in list_test_indices:
            pre_test_set.append(text)
        else:pre_train_set.append(text)
        
    return pre_train_set,pre_test_set
# get training set, test set
train_set, test_set = get_train_test_split(dataset_full,0.2)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(test_set)))


# get training set, test set, development set
new_test_set, new_dev_set = get_train_test_split(test_set,0.5)

new_train_set=train_set
random.shuffle(new_train_set)
random.shuffle(new_dev_set)
random.shuffle(new_test_set)
print ("Size training set: "+str(len(new_train_set)))
print ("Size test set: "+str(len(new_test_set)))
print ("Size development set: "+str(len(new_dev_set)))





def get_bigram_vocab(training_set):
    one_list = []
    article_tokens = []
    for instance in training_set:
        article = ','.join(instance[0])
        article_tokens.append(article)
    
    paragraph = ",".join(article_tokens)
    one_list.append(paragraph)
    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
    text_all_vec = bigram_vectorizer.fit_transform(one_list)
    
    return bigram_vectorizer

def get_bigram_vec(training_set, vectorizer):
    vec = vectorizer.transform(training_set).toarray()
    
    return vec
        
def chi_square_select(X_train,Y_train,num_features):
    fs_sentanalysis = SelectKBest(chi2, k=num_features).fit(X_train, Y_train)
    X_train_new = fs_sentanalysis.transform(X_train)
    return fs_sentanalysis, X_train_new


def mutual_info_select(X_train,Y_train,num_features):
    mfs_sentanalysis = SelectKBest(mutual_info_classif, k=num_features).fit(X_train, Y_train)
    X_train_new = mfs_sentanalysis.transform(X_train)
    return mfs_sentanalysis, X_train_new
    


# def train_svm_classifier(training_set, bigram_vectorizer, chi_num, mutual_num): # Function for training our svm classifier
#     X_train=[]
#     Y_train=[]
#     for instance in training_set:
#         vector_instance_bigram=get_bigram_vec(instance[0],bigram_vectorizer)
#         X_train.append(vector_instance_bigram)
#         Y_train.append(instance[1])
        
#     fs_sentanalysis, X_train_chi_vector = chi_square_select(X_train,Y_train,chi_num)
#     mfs_sentanalysis, X_train_mut_vector = mutual_info_select(X_train_chi_vector,Y_train,mutual_num)
    
#   # Finally, we train the SVM classifier 
#     svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
#     svm_clf.fit(X_train_mut_vector,Y_train)
#     return svm_clf, fs_sentanalysis, mfs_sentanalysis


# # print(all_text_list[-1:])
# print(all_text_list_[1313])
# print(all_label_list_[1313])


def train_svm_classifier_test(tra_set,bigram_vectorizer): # Function for training our svm classifier
    X_train=[]
    Y_train=[]
    for instance in tra_set:
        
        vector_instance_bigram=get_bigram_vec(instance[0],bigram_vectorizer)
        X_train = np.append(X_train,vector_instance_bigram)
        Y_train = np.append(Y_train,instance[1])
        

        X_train=np.asarray(X_train)
        Y_train=np.asarray(Y_train)

    
    
  # Finally, we train the SVM classifier 
    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
    svm_clf.fit(X_train,Y_train)
    return svm_clf



bigram_vec = get_bigram_vocab(dataset_full)
svm_clf = train_svm_classifier_test(new_train_set,bigram_vec)

# svm_clf, fs_sentanalysis, mfs_sentanalysis = train_svm_classifier(new_train_set, bigram_vec,500, 250)




X_test=[]
Y_test=[]
for instance in new_test_set:   #注意是test_set,上面的描述也错了
    vector_instance=get_vector_text(vocabulary,instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)


from sklearn.metrics import classification_report

Y_text_predictions = svm_clf.predict(X_test)
print(Y_text_predictions)
print(classification_report(Y_test_gold, Y_text_predictions))
